### Testing GAN

_What does our actual images look like in plot multiple images?_

Update: It looks good. It's not the Viz, it's the GAN.

In [11]:
#!pip install nibabel
import nibabel as nib
import numpy as np
import glob
import sys
import sklearn
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from functools import partial
import PIL
import PIL.Image
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

np.random.seed(42) # note that you must use the same seed to ensure consistentcy in your training/validation/testing
tf.random.set_seed(42)

In [13]:
# Load data 
df = pd.read_csv('/project/ds6050-soa2wg/team_lambda_II/ASD_DSM_CasesvsControls.csv', sep = ',')

In [14]:
df

FILE_ID  DX_GROUP  DSM_IV_TR  SEX DX_Control   DX_DSM    SEX_  \
0      Pitt_0050005         1          1    2     Autism   Autism  Female   
1      Pitt_0050006         1          1    1     Autism   Autism    Male   
2      Pitt_0050007         1          1    1     Autism   Autism    Male   
3      Pitt_0050011         1          1    1     Autism   Autism    Male   
4      Pitt_0050014         1          1    1     Autism   Autism    Male   
..              ...       ...        ...  ...        ...      ...     ...   
386  UCLA_1_0051280         2          0    1    Control  Control    Male   
387  UCLA_1_0051281         2          0    1    Control  Control    Male   
388  UCLA_1_0051282         2          0    2    Control  Control  Female   
389  UCLA_2_0051303         2          0    2    Control  Control  Female   
390  UCLA_2_0051305         2          0    2    Control  Control  Female   

                                                  PATH  
0    /project/ds6050-soa2wg/team_lambda_II/Outputs/...  
1    /project/ds6050-soa2wg/team_lambda_II/Outputs/...  
2    /project/ds6050-soa2wg/team_lambda_II/Outputs/...  
3    /project/ds6050-soa2wg/team_lambda_II/Outputs/...  
4    /project/ds6050-soa2wg/team_lambda_II/Outputs/...  
..                                                 ...  
386  /project/ds6050-soa2wg/team_lambda_II/Outputs/...  
387  /project/ds6050-soa2wg/team_lambda_II/Outputs/...  
388  /project/ds6050-soa2wg/team_lambda_II/Outputs/...  
389  /project/ds6050-soa2wg/team_lambda_II/Outputs/...  
390  /project/ds6050-soa2wg/team_lambda_II/Outputs/...  

[391 rows x 8 columns]

In [15]:
# obtain paths for all images
images_paths_f_case = list(df.query("SEX_ == 'Female' & DX_Control == 'Autism'")['PATH'])
images_paths_f_control = list(df.query("SEX_ == 'Female' & DX_Control == 'Control'")['PATH'])
images_paths_m_case = list(df.query("SEX_ == 'Male' & DX_Control == 'Autism'")['PATH'])
images_paths_m_control = list(df.query("SEX_ == 'Male' & DX_Control == 'Control'")['PATH'])

total_cases = len(images_paths_f_case) + len(images_paths_m_case)
total_controls = len(images_paths_f_control ) + len(images_paths_m_control)
# print out number of participants per category
print('There are {} female cases'.format(str(len(images_paths_f_case))))
print('---------------------------------------')
print('There are {} female controls'.format(str(len(images_paths_f_control))))
print('---------------------------------------')
print('There are {} male cases'.format(str(len(images_paths_m_case))))
print('---------------------------------------')
print('There are {} male controls'.format(str(len(images_paths_m_control))))
print('---------------------------------------')
print(f'There are {total_cases} total cases and {total_controls} total controls! {total_controls + total_cases} participants in total')
# make one giant list
#images_paths = images_paths_f_case  + images_paths_f_control + images_paths_m_case + images_paths_m_control

# positive cases only
images_paths = images_paths_f_case + images_paths_m_case

# female only (testing purposes)
#images_paths = images_paths_f_case

# male only (testing purposes)
#images_paths = images_paths_m_case

num_im = len(images_paths)
image_array = []

There are 22 female cases
---------------------------------------
There are 61 female controls
---------------------------------------
There are 124 male cases
---------------------------------------
There are 184 male controls
---------------------------------------
There are 146 total cases and 245 total controls! 391 participants in total


In [16]:
final_list = [] # create image array from paths
for path in images_paths:
    try: 
        img = nib.load(path)
        image_data = img.get_fdata()
        image_array.append(image_data)
        final_list.append(path)
    except:
        continue
    
image_array = np.asarray(image_array).astype('float32')

In [ ]:
image_array.shape

In [22]:
image_array.dtype

dtype('float32')

In [18]:
# Viz code 
def plot_multiple_images(images, n_cols=None):
    n_cols = n_cols or len(images)
    n_rows = (len(images) - 1) // n_cols + 1 #return the integer of the division 
    if images.shape[-1] == 1:
        images = np.squeeze(images, axis=-1)
    plt.figure(figsize=(n_cols, n_rows))
    for index, image in enumerate(images):
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(image[30,:,:])
        #plt.imshow(image, cmap="binary")
        plt.axis("off")

In [21]:
plot_multiple_images(image_array[0:32],8)